# Import of necessary parts for google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [2]:
from tensorboard.plugins.hparams import api as hp

In [3]:
import os
os.chdir("drive/MyDrive/datasets/dsl_data/")

In [4]:
!ls

audio		 __MACOSX		 saved_models
checkpoints	 models			 tensorboard_data
development.csv  preprocessingGoogle.py  Test_Dataset_Truncated
dsl_data.zip	 __pycache__		 Train_Dataset_Truncated
evaluation.csv	 sample_submission.csv


In [5]:
!ls drive/MyDrive

ls: cannot access 'drive/MyDrive': No such file or directory


# Necessary install of dep and import libraries

In [6]:
#!pip install git > /dev/null
#!rm ./requiremen*
#!rm ./preprocessing*
#!ls
!pip install -r ipython psutil==5.9.2 sounddevice==0.4.5 scipy==1.9.1 redis==4.3.4 tensorflow==2.10.0 tensorflow-io==0.27.0 cherrypy==18.8.0 paho-mqtt==1.6.1 > /dev/null
!pip install -r librosa tensorflow_model_optimization pandas keras tensorflow_io > /dev/null
!pip install tensorflow[io] > /dev/null
!pip install tensorflow_model_optimization > /dev/null
!pip install pydub


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'ipython'
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'librosa'
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import pandas as pd
import os
import shutil
import librosa
import numpy as np
import soundfile as sf
from concurrent.futures import ThreadPoolExecutor
import tensorflow as tf
import random
#import tensorflow_io as tfio

from tensorflow import keras
import tensorflow_model_optimization as tfmot
import argparse as ap

import sys
sys.path.append('drive/MyDrive/datasets/dsl_data/')

In [8]:
!echo "Be sure to have tensorboard, this code will be commented in final release"
#!tensorboard --logdir ../../datasets/dsl_data/tensorboard_data/ &

Be sure to have tensorboard, this code will be commented in final release


# Unzip of archive

In [9]:
!ls
!unzip -nq ./dsl_data.zip

audio		 __MACOSX		 saved_models
checkpoints	 models			 tensorboard_data
development.csv  preprocessingGoogle.py  Test_Dataset_Truncated
dsl_data.zip	 __pycache__		 Train_Dataset_Truncated
evaluation.csv	 sample_submission.csv


In [10]:
!ls

audio		 __MACOSX		 saved_models
checkpoints	 models			 tensorboard_data
development.csv  preprocessingGoogle.py  Test_Dataset_Truncated
dsl_data.zip	 __pycache__		 Train_Dataset_Truncated
evaluation.csv	 sample_submission.csv


# Code for finding optimal length of audio files

In [11]:
import os
import pandas as pd
from pydub import AudioSegment

folder_path = '../../datasets/dsl_data/audio'

def scan_folder(folder):
  duration_count = {}
  for root, dirs, files in os.walk(folder):
    for file in files:
      if file.endswith(".wav"):
        file_path = os.path.join(root, file)
        audio = AudioSegment.from_wav(file_path)
        duration = len(audio)
        if duration in duration_count:
          duration_count[duration] += 1
        else:
            duration_count[duration] = 1
  return duration_count

def create_dataframe(duration_count):
  data = {"Duration of audio file": list(duration_count.keys()), 
            "Number of audio files with that duration": list(duration_count.values())}
  df = pd.DataFrame(data)
  df = df.sort_values(by='Number of audio files with that duration', ascending=False)
  return df


# find the percentage. The duration returned in second is the size that include 1-percentage inside

def find_duration(folder_path, percentage_files=0.9):
  duration_count = {}
  for root, dirs, files in os.walk(folder_path):
    for file in files:
      if file.endswith(".wav"):
        file_path = os.path.join(root, file)
        #print(file_path)
        audio = AudioSegment.from_wav(file_path)
        duration = len(audio) / 1000 #convert from ms to sec
        if duration in duration_count:
          duration_count[duration] += 1
        else:
          duration_count[duration] = 1
    total_files = sum(duration_count.values())
    target_files = total_files * percentage_files
    current_count = 0
    for duration, count in sorted(duration_count.items()):
      current_count += count
      if current_count >= target_files:
        duration = round(duration)
        print(f"Duration of audio that makes {percentage_files*100}% of the files have that duration is: {duration} seconds")
        return duration
    



duration = find_duration(folder_path)

pathPreprocessing = "../../datasets/dsl_data/preprocessingGoogle.py"

with open(pathPreprocessing, "r") as file:
    lines = file.readlines()

# Update the 4th line
lines[8] = "length_calculated = {}\n".format(duration)


with open(pathPreprocessing, "w") as file:
    file.writelines(lines)
#print(lines)

Duration of audio that makes 90.0% of the files have that duration is: 4 seconds


In [12]:
!cat ../../datasets/dsl_data/preprocessingGoogle.py

import tensorflow as tf
# import tensorflow_io as tfio
import numpy as np
LABELS = ['change languagenone', 'activatemusic', 'deactivatelights', 'increasevolume', 'decreasevolume', 'increaseheat', 'decreaseheat']
# This is the file genarated that has the Labels that i must use for training
 
frame_length_in_s = 0.032
frame_step_in_s  = frame_length_in_s
length_calculated = 4

PREPROCESSING_ARGS = {
    'downsampling_rate': 16000,
    'frame_length_in_s': frame_length_in_s,
    'frame_step_in_s': frame_step_in_s,
}

# TRAINING_ARGS = {
#     'batch_size': 32,
#     'initial_learning_rate': 0.03,
#     'end_learning_rate': 0.001, #1.e-9,
#     'epochs': 1
# }


final_sparsity = 0.01

num_mel_bins = (int) ((16000 - 16000 * PREPROCESSING_ARGS['frame_length_in_s'])/(16000*PREPROCESSING_ARGS['frame_step_in_s']))+1
print(num_mel_bins)

PREPROCESSING_ARGS = {
    **PREPROCESSING_ARGS,
    'num_mel_bins': num_mel_bins,
    'lower_frequency': 80,
    'upper_frequency': 8000,
}

downsampling_rate 

 # Preprocessing for Train dataset files

In [13]:
import preprocessingGoogle as pr
# process_file better to be implemented here with a boolean value that checks if i am processing train_dataset or eval file
def process_file(file_path, flag):
    file_path_exists = df[df["path"] == file_path].shape[0] > 0 #flag
    if file_path_exists:
        new_sr=16000
        # identifier care
        identifier = df.loc[df["path"] == file_path, "Id"].values[0]
        identifier = str(int(identifier))
        # label constructor
        label = ""
        if flag == 1:
            label  += "_"
            action  = df.loc[df["path"] == file_path, "action"].values[0]
            object  = df.loc[df["path"] == file_path, "object"].values[0]
            label  += action + object
        # If no label available, code will just go on
        new_file_path = os.path.join(new_folder_path, identifier + label + '.wav')
        y, sr = librosa.load('../../datasets/'+file_path)
        y_truncated = librosa.effects.trim(y, top_db=50, frame_length=2048, hop_length=512, ref=np.max)[0]
        y_truncated = librosa.resample(y_truncated, orig_sr=sr, target_sr=new_sr)
        y_truncated = y_truncated[:int(pr.length_calculated*new_sr)] #if longer
        target_length = pr.length_calculated * new_sr
        y_truncated = librosa.util.fix_length(data=y_truncated, size=target_length) # padding, if shorter
        sf.write(new_file_path, y_truncated, new_sr, 'PCM_16')

31
Frame_length: 512
Frame_length: 512


In [14]:
df = pd.read_csv('../../datasets/dsl_data/development.csv', sep=',')
new_folder_path = '../../datasets/dsl_data/Train_Dataset_Truncated/'

folder_path = '../../datasets/dsl_data/'

if not os.path.isdir(new_folder_path):
  os.makedirs(new_folder_path) # hoping to have write permissions set
if not os.listdir(new_folder_path):
  with ThreadPoolExecutor() as executor: # who is your single threaddy?
    for dirpath, dirnames, filenames in os.walk(folder_path):
      dirpath = dirpath.replace("\\", "/")
      dirpath = dirpath[dirpath.index("/")+1:] # FUCK MICROSOFT AND FUCK THE FUCKING IDEA OF 
      dirpath = dirpath[dirpath.index("/")+1:] # FUCKING USING \ FOR PATH!!!!!!!!!!111oneone!!1!!!
      dirpath = dirpath[dirpath.index("/")+1:]
      for filename in filenames:
        file_path = os.path.join(dirpath, filename)
        file_path = file_path.replace("\\", "/")
        executor.submit(process_file, file_path, 1)
# print(df)
print("Execution ended")


Execution ended


 # Preprocessing for Evaluation dataset files

In [15]:
df = pd.read_csv('../../datasets/dsl_data/evaluation.csv', sep=',')
new_folder_path = '../../datasets/dsl_data/Test_Dataset_Truncated/'
folder_path = '../../datasets/dsl_data/'

if not os.path.isdir(new_folder_path):
    os.makedirs(new_folder_path)

if not os.listdir(new_folder_path):
    with ThreadPoolExecutor() as executor:
        for dirpath, dirnames, filenames in os.walk(folder_path):
            dirpath = dirpath.replace("\\", "/")
            dirpath = dirpath[dirpath.index("/")+1:]
            dirpath = dirpath[dirpath.index("/")+1:]
            dirpath = dirpath[dirpath.index("/")+1:]
            for filename in filenames:
                file_path = os.path.join(dirpath, filename)
                file_path = file_path.replace("\\", "/")
                executor.submit(process_file, file_path, 0)

print("Execution ended")

Execution ended


# Auto - updating labels

In [16]:
pathPreprocessing = "../../datasets/dsl_data/preprocessingGoogle.py"

df = pd.read_csv('../../datasets/dsl_data/development.csv', sep=',')
df['labels'] = df['action'].astype(str) + df['object'].astype(str)
distinct_values = df['labels'].unique()

result = 'LABELS = ['
for value in distinct_values:
    result += "'" + str(value) + "', "

result = result[:-2] + ']\n' # lazy workaround, the last label has a comma that is bad.. this is also bad.

with open(pathPreprocessing, "r") as file:
    lines = file.readlines()

# Update the 4th line
lines[3] = result
lines[4] = "# This is the file genarated that has the Labels that i must use for training\n"

with open(pathPreprocessing, "w") as file:
    file.writelines(lines)

# Model creation and fitting

In [17]:
parser = ap.ArgumentParser()

parser.add_argument('--batch_size', default=32, type=int, help="Choosing batch size default is 32")
parser.add_argument('--initial_learning_rate', default=0.03, type=float, help="Choosing initial_learning_rate")
parser.add_argument('--end_learning_rate', default=0.001, type=float, help="Choosing end_learning_rate")
parser.add_argument('--epochs', default=50, type=int, help="Choosing epochs")
parser.add_argument('--test_percentage', default=0.2, type=float, help="Choosing test_percentage")
parser.add_argument('--pruning_initial_step', default=0.2, type=float, help="Choosing pruning_initial_step")
parser.add_argument('--initial_sparsity', default=0.40, type=float, help="Choosing initial_sparsity")
parser.add_argument('--alpha', default=1, type=float, help="Choosing alpha")

parser.add_argument('--eval_percentage', default=0.15, type=float, help="Choosing eval_percentage")
#,'--eval_percentage','0.0'

_StoreAction(option_strings=['--eval_percentage'], dest='eval_percentage', nargs=None, const=None, default=0.15, type=<class 'float'>, choices=None, help='Choosing eval_percentage', metavar=None)

Parser arguments

In [18]:
args = parser.parse_args(['--epochs','100','--batch_size','128','--pruning_initial_step','0.9','--initial_learning_rate','0.03','--end_learning_rate','0.001','--alpha','0.125'])
# args = parser.parse_args()
num_units = 512

## This part of the code exist to manage all the folders
## Please be careful, if the directories tree is not respected, the code will not work properly

In [19]:
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
random.seed(seed)
tf.random.set_seed(seed)
np.random.seed(seed)

In [20]:
# Useful to save tensorboard data
log_dir_tensorboard = '../../datasets/dsl_data/tensorboard_data/'
if not os.path.isdir(log_dir_tensorboard):
    os.makedirs(log_dir_tensorboard)
#runs = [int(d.split('_')[1]) for d in os.listdir(log_dir_tensorboard) if 'run_' in d]
#tb_run = max(runs) + 1 if runs else 0

In [21]:
# Folder creation
train_ds_location      = '../../datasets/dsl_data/Train_Dataset_Truncated/'
log_dir_model          = '../../datasets/dsl_data/models/'
#run_{}_
model_name             = 'epochs_{}_batch_size_{}_pruning_initial_step_{}_initial_learning_rate_{}_end_learning_rate_{}_test_percentage_{}_pruning_initial_step_{}_initial_sparsity_{}_alpha_{}'.format(args.epochs,args.batch_size,args.pruning_initial_step,args.initial_learning_rate,args.end_learning_rate,args.test_percentage,args.pruning_initial_step,args.initial_sparsity,args.alpha)
checkpoint_path        = '../../datasets/dsl_data/checkpoints/' + model_name
#check_point_file_name  = checkpoint_path+'.ckpt'

In [22]:
# If folders to not exist -> create them
# This code will not check for the dataset folders, the code above must be executed
if not os.path.isdir(log_dir_model):
    os.makedirs(log_dir_model)
if not os.path.isdir(checkpoint_path):
    os.makedirs(checkpoint_path)

 # Obtaining Test data from train data, using shuffle and avoiding retaking same data on different runs

In [23]:
file_paths = []

for filename in os.listdir(train_ds_location):
    file_path = os.path.join(train_ds_location, filename)
    file_paths.append(file_path)
random.shuffle(file_paths)
num_test_files = int(len(file_paths) * args.test_percentage)
num_eval_files = int(len(file_paths) * args.eval_percentage)
#not using eval dataset


# num_eval_files = num_eval_files

# it is shuffled, so i can do this
test_paths     = file_paths[:num_test_files]                 # from 0 to num_test_files
#train_paths    = file_paths[num_test_files:]
train_paths    = file_paths[num_test_files:-num_eval_files]  # from num_test_files to end-num_eval_files
eval_paths     = file_paths[-num_eval_files:]                # until the end


In [24]:
print(len(train_paths))
print(len(test_paths))
print(len(eval_paths))

6406
1970
1478


# Preprocessing data and model creation

In [25]:
train_ds       = tf.data.Dataset.list_files(train_paths)
val_ds         = tf.data.Dataset.list_files(eval_paths)
test_ds        = tf.data.Dataset.list_files(test_paths)

train_ds       = train_ds.map(pr.preprocess).batch(args.batch_size).cache()
val_ds         = val_ds.map(pr.preprocess).batch(args.batch_size)
test_ds        = test_ds.map(pr.preprocess).batch(args.batch_size)

for example_batch, example_labels in train_ds.take(1):
  print('Batch Shape:', example_batch.shape)
  print('Data Shape:', example_batch.shape[1:])
  print('Labels:', example_labels)

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
begin_step          = int(len(train_ds) * args.epochs * args.pruning_initial_step)
end_step            = int(len(train_ds) * args.epochs)
pruning_params      = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=args.initial_sparsity,
        final_sparsity=pr.final_sparsity,
        begin_step=begin_step,
        end_step=end_step
    )
}
custom_objects      = {'PruneLowMagnitude': prune_low_magnitude}

# model_name          = 'model_'+str(args.batch_size)+'_'+str(args.alpha)
# model_name += '.h5'

hparams = {
'num_units' : num_units,
'alpha_rate': args.alpha,
'epochs'    : args.epochs,
'batch_size': args.batch_size,
}

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=example_batch.shape[1:]),
    tf.keras.layers.Conv2D(filters=int(num_units * args.alpha), kernel_size=[3, 3], strides=[2, 2],
        use_bias=False, padding='valid'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Conv2D(filters=int(num_units * args.alpha), kernel_size=[3, 3], strides=[1, 1],
            use_bias=False, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Conv2D(filters=int(num_units * args.alpha), kernel_size=[3, 3], strides=[1, 1],
        use_bias=False, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(units=len(pr.LABELS)),
    tf.keras.layers.Softmax()
    ])

model_for_pruning = prune_low_magnitude(model, **pruning_params)

Batch Shape: (128, 62, 31, 1)
Data Shape: (62, 31, 1)
Labels: tf.Tensor(
[3 6 6 5 3 3 3 0 3 0 4 3 3 3 6 4 3 4 1 3 4 3 2 3 3 1 4 3 5 6 2 3 3 6 3 4 6
 4 4 2 3 6 3 0 3 3 4 6 3 5 2 6 3 0 0 6 3 4 4 4 3 4 6 6 3 3 5 5 4 0 1 4 2 3
 4 3 6 3 3 1 3 3 1 6 3 4 1 6 6 4 4 5 0 5 0 3 0 3 3 5 3 5 0 4 0 1 6 4 3 4 0
 3 3 6 2 4 0 1 4 4 3 6 4 3 4 6 6 5], shape=(128,), dtype=int64)


In [26]:
# this model uses Transfer Learning... I mean, we transferred a model developed for another course to this course
print(example_batch.shape[1:])
model_for_pruning.summary()

(62, 31, 1)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_conv2d   (None, 30, 15, 128)      2306      
 (PruneLowMagnitude)                                             
                                                                 
 prune_low_magnitude_batch_n  (None, 30, 15, 128)      513       
 ormalization (PruneLowMagni                                     
 tude)                                                           
                                                                 
 prune_low_magnitude_re_lu (  (None, 30, 15, 128)      1         
 PruneLowMagnitude)                                              
                                                                 
 prune_low_magnitude_conv2d_  (None, 30, 15, 128)      294914    
 1 (PruneLowMagnitude)                                           
                                            

# Model fitting

In [27]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=False)

linear_decay = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=args.initial_learning_rate,
    end_learning_rate=args.end_learning_rate,
    decay_steps=len(train_ds) * args.epochs,
)
optimizer = tf.optimizers.Adam(learning_rate=linear_decay)
metrics = [tf.metrics.SparseCategoricalAccuracy()]

callbacks = [ tf.keras.callbacks.ModelCheckpoint(filepath=log_dir_tensorboard+model_name,
                                                 save_weights_only=True,
                                                 verbose=1),
             tfmot.sparsity.keras.UpdatePruningStep(), 
             keras.callbacks.TensorBoard(log_dir=log_dir_tensorboard+model_name, histogram_freq=1) , hp.KerasCallback(log_dir_tensorboard+model_name, hparams),
             ]


model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

if os.path.exists(log_dir_tensorboard+model_name+'.ckpt'):
    print("Checkpoint found, loading...")
    model.load_weights(log_dir_tensorboard+model_name+'.ckpt')
    with open(log_dir_tensorboard+model_name+"epochs.txt", "r") as file:
        contents = file.read()
        previous_epoch_run = int(contents)
        previous_epoch_run = previous_epoch_run
    print("Restoring from epoch : {}".format(previous_epoch_run))
else:
    print("No previous check_point found.")
    previous_epoch_run = 0

#validation data is test_ds validation_data=val_ds,
    
history = model.fit(train_ds, validation_data=val_ds, epochs=args.epochs, callbacks=callbacks,verbose=1,initial_epoch=previous_epoch_run) #it was valds

with open(log_dir_tensorboard+model_name+"epochs.txt", "w") as file:
    file.write(str(args.epochs))

No previous check_point found.
Epoch 1/100
51/51 [==============================] - ETA: 0s - loss: 1.8973 - sparse_categorical_accuracy: 0.2362
Epoch 1: saving model to ../../datasets/dsl_data/tensorboard_data/epochs_100_batch_size_128_pruning_initial_step_0.9_initial_learning_rate_0.03_end_learning_rate_0.001_test_percentage_0.2_pruning_initial_step_0.9_initial_sparsity_0.4_alpha_0.25
51/51 [==============================] - 67s 368ms/step - loss: 1.8973 - sparse_categorical_accuracy: 0.2362 - val_loss: 7.2949 - val_sparse_categorical_accuracy: 0.2774
Epoch 2/100
49/51 [===========================>..] - ETA: 0s - loss: 1.7680 - sparse_categorical_accuracy: 0.2640
Epoch 2: saving model to ../../datasets/dsl_data/tensorboard_data/epochs_100_batch_size_128_pruning_initial_step_0.9_initial_learning_rate_0.03_end_learning_rate_0.001_test_percentage_0.2_pruning_initial_step_0.9_initial_sparsity_0.4_alpha_0.25
51/51 [==============================] - 5s 107ms/step - loss: 1.7689 - sparse_ca

In [28]:
test_loss, test_accuracy = model.evaluate(test_ds)

training_loss = history.history['loss'][-1]
training_accuracy = history.history['sparse_categorical_accuracy'][-1]
val_loss = history.history['val_loss'][-1]
val_accuracy = history.history['val_sparse_categorical_accuracy'][-1]

print(f'Training Loss: {training_loss:.4f}')
print(f'Training Accuracy: {training_accuracy*100.:.2f}%')
print()
print(f'Validation Loss: {val_loss:.4f}')
print(f'Validation Accuracy: {val_accuracy*100.:.2f}%')
print()
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy*100.:.2f}%')

16/16 [==============================] - 5s 312ms/step - loss: 0.6840 - sparse_categorical_accuracy: 0.8142
Training Loss: 0.0091
Training Accuracy: 100.00%

Validation Loss: 0.6472
Validation Accuracy: 81.60%

Test Loss: 0.6840
Test Accuracy: 81.42%


In [29]:
with open(log_dir_model+model_name+".txt", "w") as file:
    file.write(model_name)
    file.write("\n")
    file.write("Execution lasted: " + str(args.epochs))
    file.write("\n")
    file.write(f'\nTraining Loss: {training_loss:.4f}')
    file.write(f'\nTraining Accuracy: {training_accuracy*100.:.2f}%')
    file.write("\n")
    file.write(f'\nValidation Loss: {val_loss:.4f}')
    file.write(f'\nValidation Accuracy: {val_accuracy*100.:.2f}%')
    file.write("\n")
    file.write(f'\nTest Loss: {test_loss:.4f}')
    file.write(f'\nTest Accuracy: {test_accuracy*100.:.2f}%')
    
saved_model_dir = f'./saved_models/last_model_used'
if not os.path.exists(saved_model_dir):
    os.makedirs(saved_model_dir)
model.save(saved_model_dir)

In [30]:
print(log_dir_model+model_name)
with open(log_dir_model+model_name+".txt", "r") as file:
        contents = file.read()
        print(contents)
    

../../datasets/dsl_data/models/epochs_100_batch_size_128_pruning_initial_step_0.9_initial_learning_rate_0.03_end_learning_rate_0.001_test_percentage_0.2_pruning_initial_step_0.9_initial_sparsity_0.4_alpha_0.25
epochs_100_batch_size_128_pruning_initial_step_0.9_initial_learning_rate_0.03_end_learning_rate_0.001_test_percentage_0.2_pruning_initial_step_0.9_initial_sparsity_0.4_alpha_0.25
Execution lasted: 100

Training Loss: 0.0091
Training Accuracy: 100.00%

Validation Loss: 0.6472
Validation Accuracy: 81.60%

Test Loss: 0.6840
Test Accuracy: 81.42%
